In [1]:
import pandas as pd
import numpy as np
import re
from tests import *
from matplotlib import pyplot as plt
import scipy.stats
from helpers import get_eur_price, translate_building_year_values, get_building_year_category

# Reading Data

In [2]:
dict_of_announcements = pd.read_pickle(r'parsed_announcements.pkl')
df = pd.DataFrame(dict_of_announcements)

# Creating the dependant variabile
- removing listings meant as rentals
- standardizing the prices:
    - using the same currency (EUR)
    - adding the VAT where necessary (5% for properties under 140000€ & 19% for properties over that)

In [3]:
df = df.drop(df[df['price'].apply(lambda x: '/ lună' in x)].index).reset_index(drop=True)
df['eur_price'] = df['price'].apply(lambda x: get_eur_price(x))
df['eur_price'] = np.where(
    (df['eur_price'] <= 140000) & (df['price'].apply(lambda x: 'TVA' in x)),
    df['eur_price'] * 1.05,
    np.where(
        (df['eur_price'] > 140000) & (df['price'].apply(lambda x: 'TVA' in x)),
        df['eur_price'] * 1.19,
        df['eur_price']
    )
)

In [4]:
test_rent_removed(df)
test_price_values(df)

Tests passed!
Tests passed!


# Creating independent variables

### Renaming columns
- translating from Romanian to English
- using a pythonic convention

In [5]:
original_cols = ['Nr. camere', 'Suprafaţă utilă', 'Compartimentare', 'Confort', 'Etaj', 'Nr. bucătării', 'Nr. băi', 'An construcţie', 'Structură rezistenţă', 'Tip imobil', 'Nr. balcoane', 'Nr. locuri parcare', 'Nr. garaje']
new_cols = ['rooms', 'surface_area', 'partitioning_type', 'comfort_type', 'floors', 'kitchens', 'bathrooms', 'building_year', 'building_structure', 'building_type', 'balconies', 'parking_spots', 'garages']
df.rename(columns={original_cols[i]: new_cols[i] for i in range(len(original_cols))}, inplace=True)

### Cleaning critical columns
- dropping null/invalid values
- changing to a numeric data type

In [6]:
df.drop(df[
    (df['rooms'].isnull()) | 
    (df['surface_area'].isnull()) | 
    (df['building_year'].isnull()) |
    (df['floors'].apply(lambda x: '/' not in x))
].index, inplace=True)
df['rooms'] = df['rooms'].astype(int)
df['surface_area'] = df['surface_area'].apply(lambda x: x.split(' ')[0].strip().replace(',', '.')).astype(float)

# TODO creeaza TESTE pentru: 
- valori nule
- valori acceptate in coloane

### Transforming partinion_type
The most common value ("decomandat") is by far the most numerous. To keep things simple two columns will be created:
- **partition_decomandat**
- **partition_other**

In [7]:
df['partitioning_type'].value_counts(dropna=False)

decomandat        1580
semidecomandat     867
NaN                 65
nedecomandat        20
circular            14
Name: partitioning_type, dtype: int64

In [8]:
df['partition_decomandat'] = (df['partitioning_type'] == 'decomandat').astype(int)
df['partition_other'] = (df['partitioning_type'] != 'decomandat').astype(int)

### Transforming comfort_type
Keeping the following values:
- comfort_1
- lux
- comfort_other

In [9]:
df['comfort_type'].value_counts(dropna=False)

1      1749
lux     671
NaN      96
2        24
3         6
Name: comfort_type, dtype: int64

In [10]:
df['comfort_other'] = (~df['comfort_type'].isin(['1', 'lux'])).astype(int)
df['comfort_1'] = (df['comfort_type'] == '1').astype(int)
df['comfort_lux'] = (df['comfort_type'] == 'lux').astype(int)

### Transforming apartment floor intro dummies. Final values: first_floor, last_floor, other
- splitting the x/y values into floor & max_floor
- comparing floor with max_floor to get values for dummies
- creating dummies

In [11]:
floor_series = df['floors'].apply(lambda x: x.split('/')[0]).reset_index(drop=True)
max_floor_series = df['floors'].apply(lambda x: x.split('/')[-1].strip()).astype(int).reset_index(drop=True)
floor_series = pd.Series(
        np.where(
        floor_series.apply(lambda x: any([val in x.lower() for val in ['parter', 'demi']])),
        '0',
        np.where(
            floor_series.apply(lambda x: any([val in x.lower() for val in ['ultim', 'mansard']])),
            '99',
            floor_series
        )
    )
).apply(lambda x: re.findall(r'\d+', x)[0]).astype(int)
df['floor'] = np.where(
    floor_series == 0,
    'first_floor',
    np.where(
        floor_series >= max_floor_series,
        'last_floor',
        'middle_floor'
    )
)
df['max_floor'] = max_floor_series
df = pd.concat([df, pd.get_dummies(df['floor'])], axis=1)

### Transforming bathrooms
- replacing NaNs with the most common value, 1 bathroom
- changing the data type to int
- adding the ratio between the number of bathrooms & rooms as a column

In [12]:
df['bathrooms'].value_counts(dropna=False)

1      1468
2       825
3       175
NaN      41
4        32
5         3
7         1
11        1
Name: bathrooms, dtype: int64

In [13]:
df['bathrooms'].fillna('1', inplace=True)
df['bathrooms'] = df['bathrooms'].astype(int)
df['bathrooms_ratio'] = df['bathrooms'] / df['rooms']

### Transforming building_year
- translating categories into English
- creating **not_finished** & **not_started** categories for new buildings
- assigning each year to its category
- creating dummies

In [14]:
df['building_year'] = df['building_year'].apply(lambda x: 'not_finished' if 'constructie' in x else x)
df['building_year'] = df['building_year'].apply(lambda x: 'not_started' if 'proiect' in x else x)
df['building_year'] = df['building_year'].apply(lambda x: x.replace('(finalizata)', '').strip())
df['building_year'] = df['building_year'].apply(lambda x: translate_building_year_values(x))
df['building_year'] = df['building_year'].apply(lambda x: get_building_year_category(x))
df = pd.concat([df, pd.get_dummies(df['building_year'])], axis=1)

### Transforming building_structure
- fill missing values with distinct category
- translate the most common value into English
- unify all other categories into single category
- create dummies

In [15]:
df['building_structure'].value_counts(dropna=False)

beton       1372
NaN          737
altele       298
caramida     138
lemn           1
Name: building_structure, dtype: int64

In [16]:
df['building_structure'].fillna('unknown_building_structure', inplace=True)
df['building_structure'].replace('beton', 'concrete_building_structure', inplace=True)
other_building_structures = [
    val for val in df['building_structure'].unique() 
    if val not in ['concrete_building_structure', 'unknown_building_structure']
]
df['building_structure'].replace(other_building_structures, 'other_building_structure', inplace=True)
df = pd.concat([df, pd.get_dummies(df['building_structure'])], axis=1)

### Transforming balconies
- replace null values with 0
- create new binary variable to signify the presence of at least one balcony

In [17]:
df['balconies'].value_counts(dropna=False)

1                1159
NaN               709
2                 295
1 (închise)       270
2 (închise)        48
3                  43
4                   6
5                   6
2 (1 închise)       4
3 (închise)         3
4 (2 închise)       1
4 (închise)         1
11                  1
Name: balconies, dtype: int64

In [18]:
df['has_balconies'] = (~df['balconies'].isnull()).astype(int)

### Transforming parking_spots & garages
- replace null with 0
- create new binary variable to signify the presence of at least one parking spot or garage

In [19]:
df['parking_spots'].value_counts(dropna=False)

NaN                     2080
1                        333
2                         80
1 - Inclus în preț        39
2 - Incluse în preț        4
37                         3
3                          2
5                          2
3 - Incluse în preț        1
10 - Incluse în preț       1
5 - Incluse în preț        1
Name: parking_spots, dtype: int64

In [20]:
df['garages'].value_counts(dropna=False)

NaN    2455
1        73
2        14
4         2
3         1
5         1
Name: garages, dtype: int64

In [25]:
df['has_parking_spots_or_garages'] = ((~df['garages'].isnull()) | (~df['parking_spots'].isnull())).astype(int)

### Transforming area
- creating area column with values from each url
- creating dummies

In [44]:
df['area'] = df['url'].apply(lambda x: x.split('/')[-2].replace('-', '_')+'_area')
df = pd.concat([df, pd.get_dummies(df['area'])], axis=1)

### Transforming specifications
Based on the free text available with all listings, the following features can be easily added:
- the availability of floor heating
- exclusive reliance on district heating
- the availability of video surveillence in the building

In [22]:
df['has_floor_heating'] = df['specifications'].apply(lambda x: any(['incalzire prin pardoseala' in val.lower() for val in x])).astype(int)
df['only_district_heating'] = df['specifications'].apply(lambda x: not any([('centrala proprie' in val.lower()) | ('centrala imobil' in val.lower()) for val in x])).astype(int)
df['building_with_video_surveillance'] = df['specifications'].apply(lambda x: any(['supraveghere video' in val.lower() for val in x])).astype(int)

In [46]:
df[['url', 'rooms', 'surface_area', 'bathrooms', 'partition_decomandat', 'partition_other', 'comfort_other', 'comfort_1', 'comfort_lux',
       'max_floor', 'first_floor', 'last_floor', 'middle_floor',
       'bathrooms_ratio', 'after_2010', 'before_1941', 'between_1941_1977',
       'between_1977_1990', 'between_1990_2000', 'between_2000_2010',
       'not_finished', 'not_started', 'concrete_building_structure',
       'other_building_structure', 'unknown_building_structure',
       'has_balconies', 'has_parking_spots_or_garages', 'has_floor_heating', 'only_district_heating',
       'building_with_video_surveillance', 'eur_price', '1_mai_area',
       'agronomie_area', 'aviatiei_area', 'aviatorilor_area',
       'banu_manta_area', 'chibrit_area', 'domenii_area', 'dristor_area',
       'stefan_cel_mare_area', 'titulescu_area', 'turda_area']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2546 entries, 0 to 2658
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   url                               2546 non-null   object 
 1   rooms                             2546 non-null   int32  
 2   surface_area                      2546 non-null   float64
 3   bathrooms                         2546 non-null   int32  
 4   partition_decomandat              2546 non-null   int32  
 5   partition_other                   2546 non-null   int32  
 6   comfort_other                     2546 non-null   int32  
 7   comfort_1                         2546 non-null   int32  
 8   comfort_lux                       2546 non-null   int32  
 9   max_floor                         2435 non-null   float64
 10  first_floor                       2546 non-null   uint8  
 11  last_floor                        2546 non-null   uint8  
 12  middle